In [54]:
# -*- coding: utf-8 -*-


"""train.py

This module trains LSTM text generator model.

Quated from: https://spjai.com/ai-history/
"""


## paths ---------------------------------------------------------------------
import os
import sys

MAIN_PATH = os.path.dirname(os.path.abspath(__name__))
MODELS_PATH = os.path.normpath(os.path.join(MAIN_PATH, 'models'))

sys.path.append(MAIN_PATH)


## modules ---------------------------------------------------------------------
import pathlib

import numpy as np
import cupy as cp

import chainer
import chainer.links as L
import chainer.functions as F
from chainer import Chain, Variable, datasets, optimizers
from chainer import report, training
from chainer.training import extensions
import chainer.cuda

from lstm import LSTMNet
from lstm_updater import LSTMUpdater
#from make_dataset import make_dataset


## parameters ------------------------------------------------------------------



## functions -------------------------------------------------------------------
def main():
    # params
    print('set params')
    n_lstm_layers = 1
    n_in = 200
    n_out = 200
    n_hidden = 200
    
    dev_ratio = 0.2
    batch_size = 16
    epochs = 100
    
    gpu = -1
    np.random.seed(0)
    
    if gpu >= 0:
        xp = cp
    else:
        xp = np

        
    # データの用意
    print('set data')
    print('abababa')
    x_train, x_test, y_train, y_test = mkdataset(xp, dev_ratio)
    print(len(y_train))
    n_train = len(x_train)
    n_test = len(x_test)
    print('train dataset length: {}'.format(n_train))
    print('test dataset length: {}'.format(n_test))    
    
    # モデルの宣言
    print('set model')
    rnn = LSTMNet(n_in, n_hidden, n_out)
    model = L.Classifier(rnn, lossfun=F.mean_squared_error)

    # GPU対応
    if gpu >= 0:
        chainer.cuda.get_device(0).use()
        model.to_gpu()
        print('use GPU')

    # Optimizer
    optimizer = optimizers.Adam()
    optimizer.setup(model)
    
    
    early_stopping = 20
    min_valid_loss = 1e-8
    min_epoch = 0

    train_loss, valid_loss = [], []
    print('run')
    for epoch in range(1, epochs):
        #print('epch:{} is started'.format(epoch))
        #_y = model(x=x_test, y=y_test)
        #y_pred = _y.data
        #y_pred = xp.array([1 - y, y], dtype='f').T[0]
        #print(type(y_pred), y_pred.shape)
        #y_t = xp.array(y_test)
        #print(y_t.shape)
        #accuracy = F.accuracy(y_pred, y_t).data

        #_train_loss = F.sigmoid_cross_entropy(model(x_train), y_train).data
        #_valid_loss = F.sigmoid_cross_entropy(_y, y_test).data
        #train_loss.append(_train_loss)
        #valid_loss.append(_valid_loss)

        # valid_lossが20回連続で更新されなかった時点で学習を終了
        #if min_valid_loss >= _valid_loss:
            #min_valid_loss = _valid_loss
            #min_epoch = epoch
        #elif epoch - min_epoch >= early_stopping:
            #break
            
        """以下ogino"""
        x,t = [], []
        BATCH_ROW_SIZE=100
        BATCH_COL_SIZE=100
        #for i in range(0, n_train, batch_size):
            #x_mini = x_train[i: i+batch_size]
            #y_mini = y_train[i: i+batch_size]
        for i in range(BATCH_ROW_SIZE):
            index = np.random.randint(0,n_train-BATCH_COL_SIZE+1)
            x.append(x_train[index:index+BATCH_COL_SIZE])
            t.append(y_train[index:index+BATCH_COL_SIZE])
        x = np.array(x, dtype='float32')
        t = np.array(y, dtype='float32')
        loss = 0
        total_loss = 0
        modell.reset()
        
        for i in range(BATCH_COL_SIZE):
            x_ = np.array([x[j,i] for j in range(BATCH_ROW_SIZE)],dtype='float32')[:, np.newaxis]
            t_ = np.array([t[j,i] for j in range(BATCH_ROW_SIZE)],dtype='float32')[:, np.newaxis]
            loss += model(x=x_, t=t_, train=True)
            optimizer.update()
        
            
            
            loss, accuracy = model(x=x_mini, y=y_mini, train=True)
            loss.backward()

            total_loss += loss.data
            total_accuracy += accuracy.data
            optimizer.update()
        print('epoch: {} acc: {} loss: {} valid_loss: {}'.format(
                    epoch, accuracy, _train_loss, _valid_loss))

    #loss_plot(train_loss, valid_loss)
    #serializers.save_npz('model.npz', model)


def mkdataset(xp, dev_ratio=0.2):
    # load data
    MAIN_PATH = pathlib.Path(__name__).resolve().parent
    x_file = MAIN_PATH / 'data_shaped' / 'x.npy'
    t_file = MAIN_PATH / 'data_shaped' / 't.npy'
    x = np.load(x_file)
    t = np.load(t_file)
    print(type(x), type(x[1]), type(x[1][1]))
    print(x.dtype, x[1].dtype, x[1][1].dtype)
    n_data = len(x)
    n_train = int(len(x) * (1-dev_ratio))

    # shuffle data
    idx = np.arange(n_data)
    np.random.shuffle(idx)
    x = list(x[idx])
    t = list(t[idx])

    # convert data
    x_raw = []
    t_raw = []
    if xp == cp:
        print('convert to Cupy (float32)')
        for x_sent, t_sent in zip(x, t):
            x_raw.append(cp.asarray(x_sent, dtype=cp.float32))
            t_raw.append(cp.asarray(t_sent, dtype=cp.float32))
    else:
        print('convert to Numpy (float32)')
        for x_sent, t_sent in zip(x, t):
            x_raw.append(x_sent.astype('float32'))
            t_raw.append(t_sent.astype('float32'))
    print(type(x_raw), type(x_raw[1]), type(x_raw[1][1]))
    print(x_raw[1].dtype, x_raw[1][1].dtype)
    
    # split data
    x_train = x_raw[:n_train]
    y_train = t_raw[:n_train]
    x_test = x_raw[n_train:]
    y_test = t_raw[n_train:]
    return x_train, x_test, y_train, y_test


def loss_plot(train_loss, valid_loss):
    import matplotlib.pyplot as plt
    x = np.arange(len(train_loss))
    plt.plot(x, train_loss)
    plt.plot(x, valid_loss)
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.savefig('loss.png')
    

    
    
def abst_train():
    # データの用意
    print('set data')
    MAIN_PATH = pathlib.Path(__name__).resolve().parent
    x_file = MAIN_PATH / 'data_shaped' / 'x.npy'
    t_file = MAIN_PATH / 'data_shaped' / 't.npy'
    x = np.load(x_file)
    t = np.load(t_file)
    #x, t = make_dateset()
    n_train = int(len(x) * (1-dev_ratio))
    dataset = list(zip(x, t))
    train, test = chainer.datasets.split_dataset(dataset, n_train)

    # Iterator
    print('set iterator')
    train_iter = chainer.iterators.SerialIterator(train, batch_size)
    test_iter = chainer.iterators.SerialIterator(test, batch_size, repeat=False, shuffle=False)

    # Updater &lt;- LSTM用にカスタマイズ
    print('set updater')
    updater = LSTMUpdater(train_iter, optimizer, device=gpu)

    # Trainerとそのextensions
    print('set trainer')
    trainer = training.Trainer(updater, (epochs, 'epoch'), out='results')

    # 評価データで評価
    print('eval')
    trainer.extend(extensions.Evaluator(test_iter, model, device=gpu))

    # 学習結果の途中を表示する
    print('log')
    trainer.extend(extensions.LogReport(trigger=(1, 'epoch')))

    # １エポックごとに、trainデータに対するlossと、testデータに対するlossを出力させる
    trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'validation/main/loss', 'elapsed_time']), trigger=(1, 'epoch'))

    # 学習開始
    print('run')
    trainer.run()


## execution -------------------------------------------------------------------
if __name__ == '__main__':
    main()


set params
set data
abababa
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
object float64 float64
convert to Numpy (float32)
<class 'list'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
float32 float32
2176
train dataset length: 2176
test dataset length: 545
set model
run


ValueError: setting an array element with a sequence.